Codigo que atende  a atividade 2 de Inteligencia Artificial


Alunos:Lucas Barbosa e  Pedro Peralta

link para baixar a base de dados = https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2020/INFLUD-23-11-2020.csv

Importação das bibliotecas

In [ ]:
import numpy as np #Relizar calculos numericos
import pandas as pd #Ferramente  de analise de dados 
import seaborn as sns #Biblioteca que faz graficos estatisticos
from sklearn import metrics #Constroi um relatorio com as metricas de classificação
import matplotlib.pyplot as plt #Cria as figuras para as areas de plotagem
from sklearn.neighbors import KNeighborsClassifier #Classificador que implementa a votação de K-vizinhos mais proximos
from sklearn.linear_model import LogisticRegression #Classificador de regressão logistica
from sklearn.model_selection import train_test_split #Divide  em vetores de treino e teste de forma aleatoria
from sklearn.preprocessing import StandardScaler

Leitura do arquivo 'cru' que sera nossa base de dados

In [ ]:
dados = pd.read_csv('influ.csv', encoding='UTF-8', sep=';', usecols=['IMUNODEPRE','NU_IDADE_N','GARGANTA','SATURACAO','FEBRE','DESC_RESP','FATOR_RISC','CARDIOPATI','HEMATOLOGI'  
,'SIND_DOWN','HEPATICA','ASMA','DIABETES' ,'NEUROLOGIC','PNEUMOPATI','RENAL','OBESIDADE'
,'OUT_MORBI','UTI','SUPORT_VEN','CLASSI_FIN','EVOLUCAO'])

Imprimindo o tipo de dados contidos em cada coluna

In [ ]:
dados.dtypes

Como dentro do arquivo a coluna FATOR_RISC esta no formato string e sem uma padronização, devemos converter as strings para minusculos e apos iremos  trocar o 'S' para 1 e 'N' para 2, simbolizando 1 para sim e 2 para nao

In [ ]:
dados['FATOR_RISC'] = dados['FATOR_RISC'].str.lower() #padronizando as strings para minusculo
dados['FATOR_RISC'] = dados['FATOR_RISC'].map({'s':1,'n':2})
dados['FATOR_RISC'].value_counts() #exibindo a quantidade de cada item que esta na coluna Fator_risco

Agora iremos eliminar as linhas em que a CLASSI_FIN seja diferente de 5, casos com covid-19 confirmados são marcados como 5, para isso iremos criar uma especie de filtro

In [ ]:
filtro_covid = dados['CLASSI_FIN'] == 5 
dados = dados[filtro_covid]

#para diminuir mais um pouco a quantidade de linhas do data frame iremos remover casos que evoluiram para morte mas por outro motivos eles sao demarcados como 3
filtro_evolucao = dados['EVOLUCAO'] < 3
dados = dados[filtro_evolucao]

Um dados que ha muitas marcaçoes é a coluna SUPORT_VEN, onde temos 1 para ventilação invasiva 2 para ventilação nao invasiva e 3 para nao, iremos agrupar os dados 1 com 2 e mudar 3 para 2 assim a tabela obedece a regra 1 SIM e 2 NAO

In [ ]:
dados['SUPORT_VEN'].value_counts()

In [ ]:
dados[dados['SUPORT_VEN'] == 2] = 1
dados[dados['SUPORT_VEN'] == 3] = 2
dados['SUPORT_VEN'].value_counts()

Verificando se ainda ha valores vazios dentro do Data Frame mostrando a quantidade de valores nulos

In [ ]:
dados.isnull().any()

In [ ]:
#trocando os valores vazios por 2 e subistituindo os valores 9 'iginorados' para 2 tambem removendo esses valores do Data Frame
dados.replace(9, 2, inplace=True) #determinando valores 'ignorados' para 2
dados.replace(np.nan, inplace=True) #removendo do Data Frames valores vazios
dados.dropna()
dados.isnull().any()

Apos termos apenas as linhas com casos confirmados de covid podemos eliminar esta coluna da base

In [ ]:
dados.drop(columns=['CLASSI_FIN'], axis=1, inplace=True)
#salvando nosso modelo no disco 
dados.to_csv('modelo_covid.csv', index=False)

Criando um grafico para determinar se ha muita correlação entra as tabelas e fazermos a Regressao Logistica 

In [ ]:
corr = dados.corr()
sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(13,8))
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask, 1)] = True
a = sns.heatmap(corr,mask=mask, annot=True, fmt='.2f')
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
roty = a.set_yticklabels(a.get_yticklabels(), rotation=30)

Agora iremos criar um modelo para treino e teste assin determinamos a sua acurracia

In [ ]:
#escolhendo as colunas que serao nosso valores de entrada para o modelo
X = pd.DataFrame(columns=['FATOR_RISC','SIND_DOWN','HEPATICA','ASMA','NEUROLOGIC','PNEUMOPATI','RENAL','OUT_MORBI','UTI','SUPORT_VEN', 'SATURACAO'],data=dados)

#verificando se existem valores faltantes nos dados
X.isna().any()

In [ ]:
#escolhendo a coluna do Data Frame que sera a resposta do modelo
y = pd.DataFrame(columns=['EVOLUCAO'], data = dados)

In [ ]:
#dividindo os dados de teste e treino, selecionamos 30% para os teste e os 70 para treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#normalizando os dados
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [ ]:
#verificando se ha valores vazios dentro de X
X.isna().sum()

In [ ]:
#configurando e treinando o modelo de regreção
logit =LogisticRegression(verbose=1, max_iter=1000)
logit.fit(X_train,np.ravel(y_train,order='C'))
y_pred=logit.predict(X_test)

In [ ]:
#calculando uma matriz de caonfusao, para a observar se o modelo esta correto
cnf_matrix =  metrics.confusion_matrix(y_test, y_pred)

In [ ]:
#código para melhor visualização da matriz de confusão
cnf_matrix =  metrics.confusion_matrix(y_test, y_pred)

import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
#imprimindo a matriz de confusão
plot_confusion_matrix(cnf_matrix, classes=['1','2'],
                      title='Matriz de confusão não normalizada', normalize=False)

In [ ]:
#métricas finais
print(metrics.classification_report(y_test, y_pred, target_names=['1','2']))

Utilizando a avaliaçao de comsulta com vizinhos proximos

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, random_state = 0)

In [ ]:
knn = KNeighborsClassifier(n_neighbors= 4)
knn.fit(x_train, y_train)

In [ ]:
dados['SUPORT_VEN'].value_counts()

In [ ]:
#testando modelo com entrada definida 
#neste modelo o paciente tem todos os sintomas ativos
X_new = np.array([[1,1,1,1,1,1,1,1,1,1,1]])
X_new.shape
prediction = knn.predict(X_new)
prediction

In [ ]:
knn.score(x_test, y_test)

Utilização da arvore de decisao para medir um score

In [ ]:
#dividindo os dados de teste e treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
modelo = ExtraTreesClassifier()
modelo.fit(X_train, y_train)
resultado = modelo.score(X_test, y_test)
print("Acurácia:", resultado)